In [153]:
#python.exe -m pip install --upgrade pip
#pip install bs4
#pip3 install bs4
#pip install requests
#pip install pandas
#pip install numpy

import requests
import bs4
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np 
import urllib3

urllib3.disable_warnings()

from datetime import datetime
import os

########################################
def write_file(filename, data):
    if os.path.isfile(filename):
        with open(filename, 'a') as f:          
            f.write('\n' + data)   
    else:
        with open(filename, 'w') as f:                   
            f.write(data)
  
def print_time():   
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    data = "Current Time = " + current_time
    return data
########################################

##############LINENOTI#################
def linenoti(ip_msg,status):
    #linenoti('Yo!')
    token = 'tcGa3lQMaxtkt5sGHGGdApaNIBpVtTdOZ9nfN17VipR'
    url = 'https://notify-api.line.me/api/notify'
    headers = {'Content-Type': 'application/x-www-form-urlencoded',
              'Authorization': 'Bearer ' + token}
    msg = ip_msg
    if status == 'available':
        req = requests.post(url, headers=headers, data= {'stickerPackageId': 789, 'stickerId':10858, 'message':msg})
    else:
        req = requests.post(url, headers=headers, data= {'stickerPackageId': 4, 'stickerId':274, 'message':msg})
    
    #print(req)
##############LINENOTI#################

def data_webscript():
    url='https://www.cuerp.chula.ac.th/'
    request = requests.get(url, verify=False)
    soup = bs4.BeautifulSoup(request.text,"html.parser")

    #จัดการข้อมูล
    data = soup.find_all('div',{'class':'systems-container d-flex flex-column'})
    data_alert = soup.find_all('div',{'class':'system-status outage'})
    data_title = soup.find_all('div',{'class':'system-title'})

    #จัดการข้อมูล List
    title_list = []
    alert_list = []

    j = 0
    for i in data_title:
        title = i.text.strip()
        alert = data_alert[j].text.strip()
        j += 1
        
        title_list.append(title)
        alert_list.append(alert)
        
    #จัดการข้อมูล dataframe
    df_title = pd.DataFrame([title_list]).transpose()
    df_title = df_title.rename(columns={0: 'Title'})

    df_alert = pd.DataFrame([alert_list]).transpose()
    df_title['default_alert'] = df_alert

    #ดึงวันที่ปัจจุบัน
    from datetime import datetime
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df_title['current_date'] = current_time

    df_title['CheckStatus'] = ''
    df_title['Result'] = ''

    #สร้างเงื่อนไข
    df_title['CheckStatus'] = df_title['default_alert'].apply(lambda i:i == 'ระบบขัดข้อง')
    #Default status [TRUE] คือ ระบบไม่ขัดข้อง 
    # ดังนั้นถ้าขึ้นว่า ระบบปกติ [FALSE] ให้แจ้งเตือน
    if bool(df_title['CheckStatus'][0]) or bool(df_title['CheckStatus'][1]) is True:
        df_title.loc[0,'Result'] = 'Default'
        df_title.loc[1,'Result'] = 'Default'
        print('Default')
        write_file('Default.txt' , print_time())
    else:
        df_title.loc[0,'Result'] = 'Alert'
        df_title.loc[1,'Result'] = 'Alert'
        linenoti('Alert, Please check system Fiori and S/4HANA','status')
        print('Alert')
        write_file('Alert.txt' , print_time())

    return df_title





In [82]:
import mysql.connector
import mysql

In [ ]:
#ทำการเชื่อมต่อกับฐานข้อมูลง่าย ๆ แค่ใส่ Host / User / Password ให้ถูกต้อง

connection = mysql.connector.connect(
host="localhost",
user="root",
password="password"
)
print(connection)
server_cursor = connection.cursor()

#รันสร้าง Database  ได้เลย ถ้ามีอยู่แล้วไม่ต้องสร้าง
def create_db(connection,server_cursor,db_name):
    #create_db(connection,server_cursor,'db_name4')

    sql_create = "CREATE DATABASE {}".format(db_name)
    server_cursor.execute(sql_create)
    connection.commit()

#create_db(connection,server_cursor,'webcualert')


In [154]:
#ทำการเชื่อมต่อกับฐานข้อมูลง่าย ๆ แค่ใส่ Host / User / Password / DB ให้ถูกต้อง
connection = mysql.connector.connect(
host="localhost",
user="root",
password="password",
database="webcualert"
)
db_cursor = connection.cursor()

In [85]:
 #สร้าง table ถ้ามีอยู่แล้วไม่ต้องสร้าง
def create_table(connection, db_cursor, table_name): 
    #create_table(connection, db_cursor, 'table_name')

   
    db_cursor.execute('''DROP TABLE IF EXISTS {}'''.format(table_name))
    connection.commit()

    #สร้าง Table ลงไป ก็ใช้ Execute ได้เช่นกัน
    db_cursor.execute('''CREATE TABLE IF NOT EXISTS {}( \
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY, \
        aTitle VARCHAR(255),\
        default_alert VARCHAR(255),\
        aCurrent_date datetime,\
        aCheckStatus VARCHAR(255),\
        aResult VARCHAR(255))'''.format(table_name))
    connection.commit()

#create_table(connection, db_cursor, 'webalertinfo')

In [123]:
def insert_data(connection, db_cursor, df_msg,n):   
    #insert_data(connection, db_cursor, df_msg,index)
    #สร้าง String ไว้รอใส่คำสั่งได้เลย
    sql_command = "INSERT INTO webalertinfo (\
                    id,aTitle, default_alert, aCurrent_date, aCheckStatus, aResult) \
                    VALUES (%s, %s, %s, %s, %s, %s)"

    #Value ที่ต้องการใส่ใน Command ทำไว้ในรูปแบบ Tuple ไว้ทำการ map กับคำสั่งด้านบนในตรง VALUES
    #val = (None,"Hi3", "Hi", "2024-08-27 09:05:18", "Hi", "Hi")
    tt = df_msg['Title'][n]
    df = df_msg['default_alert'][n]
    cd = df_msg['current_date'][n]
    cs = df_msg['CheckStatus'][n]
    rs = df_msg['Result'][n]

    val = None, tt, df, cd, cs, rs

    #สั่งให้คำสั่งทำงานได้เลย
    db_cursor.execute(sql_command, val)
    connection.commit()

In [133]:
#initial insert value
for index , row in df_data_out.iterrows():
    insert_data(connection, db_cursor, df_data_out,index)

connection.commit()
#แสดงว่ามีกี่แถวที่ทำงานสำเร็จ
#print(db_cursor.rowcount, "Succeed !")

In [134]:
#create update function

def update_data(connection, db_cursor, df_msg,n):
    #update_service(conn, cursor, df_name, 0)
    tt = df_msg['Title'][n]
    df = df_msg['default_alert'][n]
    cd = df_msg['current_date'][n]
    cs = df_msg['CheckStatus'][n]
    rs = df_msg['Result'][n]

    sql = '''UPDATE webalertinfo 
    SET aTitle = %s , 
    default_alert = %s,
    aCurrent_date = %s,
    aCheckStatus = %s,
    aResult = %s
    WHERE aTitle = %s '''
    val = tt, df, cd, cs, rs, tt

    db_cursor.execute(sql, val)
    connection.commit()  


In [184]:
df_data = data_webscript()
for index , row in df_data.iterrows():
    update_data(connection, db_cursor, df_data,index)

Default


In [185]:
#Select
#ฟังก์ชั่น fetchall() ฟังก์ชั่นสำหรับการแสดงผลข้อมูลจาก database
db_cursor.execute('''SELECT aTitle, aResult FROM webalertinfo''')
myresult = db_cursor.fetchall()
connection.commit()

aTitle_list_t2 = []
aResult_list_t2 = []
for i in myresult:
    aTitle_t2 = i[0]
    aTitle_list_t2.append(aTitle_t2)

    aResult_t2 = i[1]
    aResult_list_t2.append(aResult_t2)


df_Query = pd.DataFrame()
df_Query['Title'] = aTitle_list_t2
df_Query['aResult_t2'] = aResult_list_t2


In [186]:
df_data_merge = df_data.merge(df_Query, how='right') 
df_data_merge

,Title,default_alert,current_date,CheckStatus,Result,aResult_t2
0,Fiori,ระบบขัดข้อง,2024-09-22 21:15:16,True,Default,Default
1,S/4HANA,ระบบขัดข้อง,2024-09-22 21:15:16,True,Default,Default


In [188]:
#Check status change
for index , row in df_data_merge.iterrows():
    if row['Result'] == row['aResult_t2']:
        #print('yes')
        df_data_merge['StausChange'] = 'StatusNotChange'
    elif row['Result'] != row['aResult_t2']:
        #print('yess')
        df_data_merge['StausChange'] = 'StatusChange'


In [189]:
import requests

def linenoti(ip_msg,status):
    #linenoti('Yo!')
    token = 'OuPQz7cOOAsPitWXXTqZ4LAP1QHCCRIqjhWckMYlgWm'
    url = 'https://notify-api.line.me/api/notify'
    headers = {'Content-Type': 'application/x-www-form-urlencoded',
              'Authorization': 'Bearer ' + token}
    msg = ip_msg
    if status == 'available':
        req = requests.post(url, headers=headers, data= {'stickerPackageId': 789, 'stickerId':10858, 'message':msg})
    elif status == 'Alert':
        req = requests.post(url, headers=headers, data= {'stickerPackageId': 4, 'stickerId':274, 'message':msg})
    
    #print(req)

In [190]:
for index , row in df_data_merge.iterrows():
    #print(index)
    #print(row['Title'])
    #print(row['Result'])
    service_name = row['Title']
    status = row['Result']
    ChackChange = row['StausChange']

    print(ChackChange)
    print(status)

    if ChackChange == 'StatusChange' and status == 'Alert':
        msg = 'Service name: ' + service_name + 'has **ALERT**'
        linenoti(msg,'Alert')
    elif ChackChange == 'StatusChange' and status == 'Default':
        msg = 'Service name: ' + service_name + ' **CLEAR**'
        linenoti(msg,'available')

StatusNotChange
Default
StatusNotChange
Default
